sesionの数、yadの数、locationの数を確認して、sessionとyadでCFがうまくいかなかった理由とlocationでうまくいきそうかを考える

In [1]:
%cd ..

/kaggle/working


In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../experiments/ensemble_001"):
    cfg = compose(config_name="config.yaml", overrides=["exp=base"])
    print(OmegaConf.to_yaml(cfg))

debug: false
seed: 42
dir:
  data_dir: /kaggle/working/input/atmaCup16_Dataset
  output_dir: /kaggle/working/output
  exp_dir: /kaggle/working/output/exp
  cand_unsupervised_dir: /kaggle/working/output/cand_unsupervised
  cand_supervised_dir: /kaggle/working/output/cand_supervised
  datasets_dir: /kaggle/working/output/datasets
exp:
  other_dirs:
  - output/exp/008_split/base
  first_dirs:
  - output/exp/008_split/first



In [5]:
import logging
import os
import pickle
import sys
import time
from pathlib import Path

import hydra
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf
from tqdm.auto import tqdm

import utils
import wandb
from utils.load import load_label_data, load_log_data, load_session_data, load_yad_data
from utils.logger import get_logger
from utils.metrics import calculate_metrics

In [6]:
yado_df = load_yad_data(Path(cfg.dir.data_dir))
yado_df.shape

(13806, 13)

In [8]:
train_log_df = load_log_data(Path(cfg.dir.data_dir), "train")
len(train_log_df['session_id'].unique())

288698

In [14]:
train_log_df.group_by(["session_id"]).agg(pl.col("seq_no").max())[
    "seq_no"
].value_counts().sort(by="seq_no")

seq_no,counts
i64,u32
0,185386
1,82793
2,15350
3,4025
4,833
5,223
6,65
7,18
8,4
